# OpenEO

## Overview

This document serves as a quick reference guide of how to access OpenEO's APIs, using Python.
This notebook requires some libraries which aren't included in the Jupyter environment, please be sure to follow the readme to create an environment and install such libraries in a correct way.

***Steps***:
1. Preparation
2. Browse Collections
3. Operate on Datacubes
4. Visualization

## Preparation
Firstly we need to install the `openeo` library and import it:

In [ ]:
pip install openeo rasterio matplotlib

In [ ]:
import openeo
import json

In [ ]:
BASE_URL="https://foodsecurity-explorer.openeo.insula.earth/"
conn=openeo.connect(BASE_URL)
conn.authenticate_oidc()

If the openeo library throws an error between two different sessions while authenticating then execute this snippet as a workaround and try again:

In [ ]:
import os
os.remove("/home/jovyan/.local/share/openeo-python-client/refresh-tokens.json")

## Browse Collections

The OpenEO client is very easy to use and automatically takes care of what we need to do. For example this is what we need to do to get and browse all available collections:

In [ ]:
collections = conn.list_collections()
j=json.dumps(collections)

In [ ]:
#visual list view
collections

You can also get the collection lists by ID with the following command:

In [ ]:
ids = conn.list_collection_ids()

In [ ]:
#IDs only:
ids

In [ ]:
print("Available Processes")
process_ids = [process["id"] for process in conn.list_processes()]
print(process_ids)

## Operate on Datacubes

To retrieve the datacube we need an **AoI**, a **time interval** in days, a set of **bands** and obviously the **collection** from which we get it. In this example we are getting a datacube around the island of Pellworm.

In [ ]:
bracciano = {"west": 12.200, "south": 42.120, "east": 12.300, "north": 42.180}
t = ["2024-01-05", "2024-12-10"]

dc = conn.load_collection(
    "ecostress_l2t_stars",
    spatial_extent = bracciano,
    temporal_extent = t,
    bands = ["NDVI"]
)

print(dc.to_json())

To operate on the datacubes there are various different ways. The easier one is to use the various functions exposed by the datacubes which can be consulted at the [datacube documentation](https://open-eo.github.io/openeo-python-client/api.html#module-openeo.rest.datacube). For this example we will use NDVI.
Moreover after applying all processes you want to execute, we need to tell the back-end to export the datacube, for example as GeoTiff.

In [ ]:
dc = dc.min_time()
dc = dc.save_result(format="gtiff")

It's important to note that what it has been done until now it's only a definition, to start the processor and get what we asked we need to create and run a job:

In [ ]:
job = dc.create_job(title="my_ndvi")

In [ ]:
job.start_and_wait()
job.get_results().download_files("output")

## Visualization

This section is out of the scope of openEO, through these libraries we can open and see the data we have just downloaded:

In [ ]:
import rasterio
from rasterio.plot import show, show_hist
from rasterio.io import MemoryFile
import matplotlib.pyplot as plt

In [ ]:
img = rasterio.open("output/result.tiff")

fig,(axr,axl) = plt.subplots(1,2, figsize=(9, 4))
axr.set_xlim(0,1)
show_hist(img,bins=255,ax=axr)
show(img,ax=axl,cmap=plt.cm.RdYlGn,title='NDVI',vmin=0,vmax=1)